In [ ]:
### Observations:
# https://www.paepper.com/blog/posts/depthwise-separable-convolutions-in-pytorch/
# FLOPs = 2x Number of Kernel x Kernel Shape x Output Shape
# I'm 90% sure I won't get able to see the actual source code because it'll be in C++.
# First weights dimension is output dimension, second is input dimension // groups, third dimension is kernel size, and the first dimension has to be a mulltiple of the number of groups.
# In the case where all three parameters are equal to model_dim, I Think the weights are simply a stack of [model_dim] one-channel kernels.
# For me, a more intuitive way to parameterize instead of "groups" is n_kernels; i.e. number of kernels per input channel; i.e. input_dim // groups

In [12]:
import torch 
import torch.nn as nn
class SeqConv(nn.Module):
    
    
    def __init__(self, model_dim, kernel_size, groups = None):
        super().__init__()
        self.model_dim = model_dim
        self.kernel_size = kernel_size
        self.groups = groups if groups is not None else model_dim
        self.conv = nn.Conv1d(model_dim, model_dim, kernel_size, padding = kernel_size - 1, groups = self.groups)

    def forward(self, x):
        b, l, d = x.shape
        x = x.transpose(1, 2)
        x = self.conv(x)[..., :l]
        x = x.transpose(1, 2)
        return x


def conv_wrap(conv, x):
    b, l, d = x.shape
    x = x.transpose(1, 2)
    x = conv(x)[..., :l]
    x = x.transpose(1, 2)
    return x

In [7]:
# no gradients
torch.set_grad_enabled(False)
model_dim = 4
input_dim = model_dim*2
output_dim = model_dim
#groups = model_dim
groups = 2
kernel_size = 3
seq_len = 4
#x = torch.randn(1, seq_len, model_dim)
#x = torch.tensor([[[1, 1.1], [2, 2.1], [3, 3.1], [4, 4.1]]], dtype = torch.float32)
x = torch.tensor([[[1, 1.1, 1.01, 1.001], [2, 2.1, 2.01, 2.001], [3, 3.1, 3.01, 3.001], [4, 4.1, 4.01, 4.001]]], dtype = torch.float32)
#print(x.shape)
conv = nn.Conv1d(input_dim, output_dim, kernel_size, padding = kernel_size - 1, groups = groups, bias = False)
print(conv.weight.shape)
#w = torch.tensor([[[1, 2, 3]], [[-1, -2, -3]]], dtype = torch.float32)
#w = torch.tensor([[[1, 2, 3], [0.1, 0.2, 0.3]], [[-1, -2, -3], [-0.1, -0.2, -0.3]]], dtype = torch.float32)
w = torch.tensor([[[1, 2, 3], [0.1, 0.2, 0.3]], [[0.2, 0.4, 0.6], [0.11, 0.22, 0.33]], [[-1, -2, -3], [-0.1, -0.2, -0.3]], [[-0.2, -0.4, -0.6], [-0.11, -0.22, -0.33]]], dtype = torch.float32)
print(w.shape)
#assert w.shape == conv.weight.shape
conv.weight = nn.Parameter(w)
out = conv_wrap(conv, x)
print(out.shape)
# the first output channel...looks like it was 0 + 0 + 3*1.  Now with groups = 1...changing the second set of weights doesn't affect this...changing the second channel of the input activations does.
# so then what must be happening is that each output channel is the product of one kernel with all the input channnels.
# Therefore, the first position of the output should be 0 + 0 + 3*1 + 0.3*1.1?  Okay, so each input channel gets its own kernel for each output channel?  That sounds right.
# "groups" describes how we group up the output channels and align them with input channels.  So with groups = 2, each input channel gets two kernels (output_dim // groups) and each output channel gets two input channels (input_dim // groups).
# What's another way we could parameterize this, then? n_kernels? n_kernels = input_dim // groups?
out[...,0] 


torch.Size([4, 4, 3])
torch.Size([4, 2, 3])
torch.Size([1, 4, 4])


tensor([[ 3.3300,  8.8500, 15.4600, 22.0600]])

In [89]:
# no gradients
torch.set_grad_enabled(False)
for i in range(10):
    n_kernels = torch.randint(1, 10, (1,)).item()
    in_groups = torch.randint(1, 10, (1,)).item()
    out_groups = torch.randint(1, 10, (1,)).item()
    input_dim = in_groups * n_kernels
    output_dim = in_groups * out_groups # okay this works.
    
    groups = in_groups
    kernel_size = 3
    seq_len = 5
    x = torch.randn(1, seq_len, input_dim)
    conv = nn.Conv1d(input_dim, output_dim, kernel_size, padding = kernel_size - 1, groups = groups, bias = False)
    assert conv.weight.shaoe[0] == output_dim
    assert conv.weight.shape[1] == n_kernels
    assert conv.weight.shape[2] == kernel_size
    out = conv_wrap(conv, x)
    print(f" n_kernels: {n_kernels}, in_groups: {in_groups}, out_groups: {out_groups}, input {x.shape}, output: {out.shape}, weights {conv.weight.shape}")


# Okay so what are we saying here?
# The first example takes 24 input channels, splits them into 3 groups of 8, using 8 kernels.  What happens with these groups, then?  I think it takes the 3 groups and either scales them up by 4, or recombines them each in 4 different ways.
# Let's tackle the second example.  There are 9 kernels, each of which gets applied to all of the input channels.  So at some intermediate stage, there are 9 numbers.  At the end, there are 5.  So I feel like that must be scaling; there's no simple way to divide 9 up into 5.
# Anyway I think I have come about as far as I can exploring from this angle; now let's read about separable convolutions.

 n_kernels: 8, in_groups: 3, out_groups: 4, input torch.Size([1, 5, 24]), output: torch.Size([1, 5, 12]), weights torch.Size([12, 8, 3])
 n_kernels: 9, in_groups: 1, out_groups: 5, input torch.Size([1, 5, 9]), output: torch.Size([1, 5, 5]), weights torch.Size([5, 9, 3])
 n_kernels: 2, in_groups: 2, out_groups: 5, input torch.Size([1, 5, 4]), output: torch.Size([1, 5, 10]), weights torch.Size([10, 2, 3])
 n_kernels: 8, in_groups: 2, out_groups: 4, input torch.Size([1, 5, 16]), output: torch.Size([1, 5, 8]), weights torch.Size([8, 8, 3])
 n_kernels: 4, in_groups: 7, out_groups: 9, input torch.Size([1, 5, 28]), output: torch.Size([1, 5, 63]), weights torch.Size([63, 4, 3])
 n_kernels: 6, in_groups: 7, out_groups: 9, input torch.Size([1, 5, 42]), output: torch.Size([1, 5, 63]), weights torch.Size([63, 6, 3])
 n_kernels: 6, in_groups: 3, out_groups: 5, input torch.Size([1, 5, 18]), output: torch.Size([1, 5, 15]), weights torch.Size([15, 6, 3])
 n_kernels: 5, in_groups: 6, out_groups: 1, inp

In [ ]:

input_dim = n_kernels * in_scale
    
groups = model_dim
n_kernels = 2
groups = in_scale
kernel_size = 3
seq_len = 5
x = torch.randn(1, seq_len, input_dim)
print(x.shape)
conv = nn.Conv1d(input_dim, output_dim, kernel_size, padding = kernel_size - 1, groups = groups, bias = False)
print(conv.weight.shape)
out = conv_wrap(conv, x)
print(out.shape)

In [7]:
# https://www.paepper.com/blog/posts/depthwise-separable-convolutions-in-pytorch/
# They start by noting the flops for a standard 2D convolution...
# W * H * K * K * Ci * Co...is that equivalent to 2x Number of Kernel x Kernel Shape x Output Shape?
# K*K is the kernel shape, W*H is the output shape, so I guesss 2x the number of kernels must be Ci*Co?  That doesn't seem quite right to me...
# Number of parameters is K*K*Ci*Co, so it's actually more expensive than just a matrix multiplication.
# And I'm guessing a standard convolution is basically groups = 1?
import torch 
import torch.nn as nn
model_dim = 7
input_dim = model_dim
output_dim = model_dim
groups = model_dim
#groups = 1
kernel_size = 3
seq_len = 4
conv = nn.Conv1d(input_dim, output_dim, kernel_size, padding = kernel_size - 1, groups = groups, bias = False)
print(conv.weight.shape) # So yeah, that looks like it's basically kernel size times the model dimension squared.  Like a dense layer, but with a kernel size.
x = torch.randn(1, seq_len, model_dim)
print(x.shape)
out = conv_wrap(conv, x)
print(out.shape) # with groups = 1, we go from 1, 4, 7 to 1, 4, 7.  With groups = model_dim, we do exactly the same thing but with 1/model_dim as many parameters.count

torch.Size([7, 1, 3])
torch.Size([1, 4, 7])
torch.Size([1, 4, 7])


In [8]:
# Okay, so this is pretty simple:
# A "depthwise convolution" is what we've been doing so far, where each input channel gets its own kernel for each output channel, and no scaling occurs.
# A "pointwise convolution" is basically the same thing as a dense layer, but applied over the sequential dimension.  That's probably why the Hugging Face implementation of GPT-2 attention uses Conv1d instead of Linear.
# A "separable convolution" is just a depthwise convolution followed by a pointwise convolution.  In a sense, Transformers already kind of do conceptually the same thing, insofar as the state mixers and sequence mixers are separate steps.
# So the number of parameters in a sequence convolution that doesn't scale is kernel_size * model_dim * model_dim.  The number of parameters in a sequence convolution that does scale is kernel_size * model_dim * model_dim.
# I think that's actually quite a lot for long kernels; I think that attention layers have only 4*model_dim*model_dim parameters.
# Wait a second...No.  Totally wrong.  There are only kernel_size*model_dim parameters when groups = model_dim.  So in that case, all that really matters is whether the kernel size is longer than the model dimension, which it may or may not be.
# And the FLOPs here are supposed to be tokens*model_dim*model_dim + tokens*model_dim*kernel_size. 
# Oh right, the whole reason attention is a resource issue; it's tokens*tokens*model_dim.
# https://medium.com/@pashashaik/a-guide-to-hand-calculating-flops-and-macs-fa5221ce5ccc
# this says it's 2 * tokens * kernel_size * model_dim, which is what the other source said.
# torchprofile might be useful for this....profile_macs, profile_flops?

torch.Size([7, 1, 3])


In [14]:
#!pip install torchprofile
from torchprofile import profile_macs # a MAC is essentially two FLOPs
model_dim = 7
kernel_size = 3
seq_len = 4
seq_conv = SeqConv(model_dim, kernel_size)
x = torch.randn(1, seq_len, model_dim)
profile_macs(seq_conv, x) # this i supposedly 2*4*3*7 = 168, but it's returning 126.  Oh, I think that's the padding; if we shave one off the sequence length it works out correctly.


126

In [16]:
2*3*3*7

126